In [124]:
from mpl_toolkits.mplot3d import Axes3D  
# Axes3D import has side effects, it enables using projection='3d' in add_subplot
import matplotlib.pyplot as plt
import numpy as np
import random
from IPython.display import HTML, display

def getFunction(x):
    a = x[0]
    b = x[1]
    r = np.sin(a**2 - b**2) + np.cos(a**2 + b**2)
    return r

def is_pos_def(x):
    if np.array_equal(x, x.T):
        try:
            np.linalg.cholesky(x)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False

def getGradient(x):
    a = x[0]
    b = x[1]
    r0 = 2*a*(np.cos(a**2 - b**2) - np.sin(a**2 + b**2))
    r1 = -2*b*(np.cos(a**2 - b**2) + np.sin(a**2 + b**2))
    r = np.array([[r0], 
                 [r1]])
    return r

def getHessian(x):
    a = x[0]
    b = x[1]
    r0 = -2*(2*a**2*np.sin(a**2 - b**2) + np.sin(a**2 + b**2) + 2*a**2*np.cos(a**2 + b**2) - np.cos(a**2 - b**2))
    r1 = 4*a*b*(np.sin(a**2 - b**2) - np.cos(a**2 + b**2))
    r2 = 4*a*b*(np.sin(a**2 - b**2) - np.cos(a**2 + b**2))
    r3 = -2*(2*b**2*np.sin(a**2 - b**2) + np.sin(a**2 + b**2) + 2*b**2*np.cos(a**2 + b**2) + np.cos(a**2 - b**2))
    r = np.array([[r0, r1], [r2, r3]])
    return r

def armijo(x, d):
    y = 0.8
    n = 0.25
    t = 1
    
    gradient = getGradient(x)
    num_calls = 0
    while getFunction([x[0] + t*d[0,0], x[1] + t*d[1,0]]) > getFunction(x) + n * t * (gradient.transpose().dot(d)):
        t = y*t
        num_calls += 1
    
    return t, num_calls

def auxAurea(x, d, t):
    return getFunction([x[0] + t*d[0,0], x[1] + t*d[1,0]])
    
def aurea(x, d):
    e = 0.001
    p = 0.001
    teta1 = (3 - 5**(1/2))/2
    teta2 = 1 - teta1
    
    # Obtendo Intervalo [a,b]
    a = 0
    s = p
    b = 2*p
    while auxAurea(x, d, b) < auxAurea(x, d, s):
        a = s
        s = b
        b = 2*b
        
    # Obtendo t em [a,b]
    u = a + teta1 * (b - a)
    v = a + teta2 * (b - a)
    num_calls = 0
    while (b - a) > e:
        if auxAurea(x, d, u) < auxAurea(x, d, v):
            b = v
            v = u
            u = a + teta1 * (b - a)
        else:
            a = u
            u = v
            v = a + teta2 * (b - a)
        num_calls += 1
    
    return (u + v)/2, num_calls

def inverseMatrix(x):
    a,b,c,d = x
    det = a * d - b * c
    x[0] = d/det
    x[1] = -1*b/det
    x[2] = -1*c/det
    x[3] = a/det
    return

def minGradient(xi, search):
    it = 0
    x = xi
    grad = getGradient(x)
    p = np.array([[10**10], 
                 [10**10]])
    sum_num_calls = 0
    while (it < 1000 
           and (abs(grad[0,0]) > 10**-15 or abs(grad[1,0]) > 10**-15)
           and (abs(p[0,0]) > 10**-8 or abs(p[1,0]) > 10**-8)):
            d = np.array([[-1 * grad[0,0]], 
                          [-1 * grad[1,0]]])
            t, num_calls = search(x, d)
            sum_num_calls += num_calls
            x0 = x
            x = [x[0] + t*d[0,0], x[1] + t*d[1,0]]
            p = np.array([[x[0] - x0[0], x[1] - x0[1]]]).transpose()
            it = it+1
            grad = getGradient(x)
    
    return x, it, sum_num_calls

def minNewton(xi, search):
    it = 0
    x = xi
    d = [0,0]
    grad = getGradient(x)
    p = np.array([[10**10], 
                 [10**10]])
    sum_num_calls = 0
    while (it < 1000 
           and (abs(grad[0,0]) > 10**-15 or abs(grad[1,0]) > 10**-15)
           and (abs(p[0,0]) > 10**-10 or abs(p[1,0]) > 10**-10)):
            hess = getHessian(x)
            if (is_pos_def(hess)):
                invhess = np.linalg.inv(hess) 
                grad = getGradient(x)
                d = -1*invhess.dot(grad)
                t, num_calls = search(x, d)
                sum_num_calls += num_calls
                x0 = x
                x = [x[0] + t*d[0,0], x[1] + t*d[1,0]]
                p = np.array([[x[0] - x0[0], x[1] - x0[1]]]).transpose()
                it = it+1
            else:
                if (it == 0):
                    print(xi, 'initial hess not pos def')
                break
    
    return x, it, sum_num_calls

def minQuasiNewton(xi, search):
    it = 0
    x = xi
    d = [0,0]
    h = np.array([[1,0],[0,1]])
    gradx0 = getGradient(x)
    q = np.array([[10**10], 
                 [10**10]])
    p = np.array([[10**10], 
                 [10**10]])
    sum_num_calls = 0
    while (it < 1000 
           and (abs(gradx0[0,0]) > 10**-15 or abs(gradx0[1,0]) > 10**-15)
          and (abs(p[0,0]) > 10**-4 or abs(p[1,0]) > 10**-4)):
            d = -1*(h.dot(gradx0))
            t, num_calls = search(x, d)
            sum_num_calls += num_calls
            x0 = x
            x = [x[0] + t*d[0,0], x[1] + t*d[1,0]]
            p = np.array([[x[0] - x0[0], x[1] - x0[1]]]).transpose()
            gradx = getGradient(x)
            q = gradx - gradx0
            gradx0 = gradx
            part1 = ((q.transpose().dot(h)).dot(q))[0,0] / ((p.transpose()).dot(q))[0,0]
            part2 = p.dot(p.transpose()) / (p.transpose().dot(q))[0,0]
            part3 = ((p.dot(q.transpose())).dot(h) + h.dot(q).dot(p.transpose())) / ((p.transpose()).dot(q))[0,0]
            h = h + ((1 + part1)* part2) - part3
            it = it+1
    
    return x, it, sum_num_calls

def getResults(x, minimize, search):
    r, i, num_calls = minimize(x, search)
    m = getFunction(r)
    return [x, i, num_calls, r, m, -2-m]
        

        
        
xo_list = [
    [-1.8385400365595483, 4.359940932133885], 
    [8.928601514360016, 3.3197980530117723], 
    [0.7630828937395717, 7.7991879224011464], 
    [0.5623534090613251, 4.536183806757618], 
    [0.10374153885699955, 5.018745921487388]]
# i = 0 
# while i < 15:
#     xo_list.append([random.uniform(-10.0, 10.0),random.uniform(-10.0, 10.0)])
#     i+=1
minimization_methods = [minGradient, minNewton, minQuasiNewton]
minimization_names = ["gradiente", "newton", "quase newton"]
search_methods = [armijo, aurea]
search_names = ["armijo", "aurea"]

for method in minimization_methods:
    for search in search_methods:
        data = [["X0", "Iter.", "Search Iter.", "X Opt.", "Opt. Value", "Error"]]
        for xo in xo_list:
            data.append(getResults(xo, method, search))
            
        print("Usando o método de minimização %s with the search method %s" %(minimization_names[minimization_methods.index(method)], search_names[search_methods.index(search)]))
        display(HTML('<table><tr>{}</tr></table>'.format(
           '</tr><tr>'.join(
               '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )))
            
        
# data = [["X0", "Iter.", "Search Iter.", "X Opt.", "Opt. Value", "Error"],
#          ["Sun",696000,1989100000],
#          ["Earth",6371,5973.6],
#          ["Moon",1737,73.5],
#          ["Mars",3390,641.85]]


# display(HTML('<table><tr>{}</tr></table>'.format(
#        '</tr><tr>'.join(
#            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
#        )))


Usando o método de minimização gradiente with the search method armijo


X0,Iter.,Search Iter.,X Opt.,Opt. Value,Error
"[-1.8385400365595483, 4.359940932133885]",37,19,"[-1.9816636407159987, 4.250195021564859]",-1.9999999999999978,-2.220446049250313e-15
"[8.928601514360016, 3.3197980530117723]",51,26,"[8.906470368326048, 3.4323421048746736]",-1.9999999999999774,-2.2648549702353193e-14
"[0.7630828937395717, 7.7991879224011464]",492,24,"[0.8862266473483452, 7.674950315095602]",-1.99999999999975,-2.5002222514558525e-13
"[0.5623534090613251, 4.536183806757618]",209,19,"[0.8862268421315791, 4.604970191073439]",-1.9999999999999758,-2.4202861936828413e-14
"[0.10374153885699955, 5.018745921487388]",190,19,"[0.886227012482084, 4.604970191469733]",-1.9999999999999734,-2.6645352591003757e-14


Usando o método de minimização gradiente with the search method aurea


X0,Iter.,Search Iter.,X Opt.,Opt. Value,Error
"[-1.8385400365595483, 4.359940932133885]",32,1,"[-1.9816636301001327, 4.250195028506188]",-1.999999999999994,-5.995204332975845e-15
"[8.928601514360016, 3.3197980530117723]",49,1,"[8.906470369974187, 3.432342112723027]",-1.999999999999992,-7.993605777301127e-15
"[0.7630828937395717, 7.7991879224011464]",248,1,"[0.8862266239781644, 7.674950310933431]",-1.999999999999714,-2.859934511434403e-13
"[0.5623534090613251, 4.536183806757618]",194,1,"[0.886226821453492, 4.604970181960542]",-1.999999999999965,-3.5083047578154947e-14
"[0.10374153885699955, 5.018745921487388]",133,1,"[0.8862270707483276, 5.242989192153612]",-1.9999999999999312,-6.88338275267597e-14


([0.7630828937395717, 7.7991879224011464], 'initial hess not pos def')
([0.10374153885699955, 5.018745921487388], 'initial hess not pos def')
Usando o método de minimização newton with the search method armijo


...

X0,Iter.,Search Iter.,X Opt.,Opt. Value,Error
"[-1.8385400365595483, 4.359940932133885]",14,1,"[-1.9816636579485287, 4.250195037321147]",-1.9999999999999893,-1.0658141036401503e-14
"[8.928601514360016, 3.3197980530117723]",14,1,"[8.906470369359976, 3.432342129688861]",-1.999999999999994,-5.995204332975845e-15
"[0.7630828937395717, 7.7991879224011464]",0,0,"[0.7630828937395717, 7.7991879224011464]",0.6747782211720822,-2.6747782211720823
"[0.5623534090613251, 4.536183806757618]",15,1,"[0.8862269120195889, 4.604970191007048]",-1.9999999999999971,-2.886579864025407e-15
"[0.10374153885699955, 5.018745921487388]",0,0,"[0.10374153885699955, 5.018745921487388]",0.9535412823416378,-2.9535412823416376


Usando o método de minimização quase newton with the search method armijo


X0,Iter.,Search Iter.,X Opt.,Opt. Value,Error
"[-1.8385400365595483, 4.359940932133885]",7,0,"[-1.9816624509873528, 4.25019405201015]",-1.999999999908931,-9.106893017474249e-11
"[8.928601514360016, 3.3197980530117723]",7,0,"[8.906470318519741, 3.4323422029006263]",-1.999999999998763,-1.2370104940373494e-12
"[0.7630828937395717, 7.7991879224011464]",8,0,"[0.8862269117344291, 7.67495029531456]",-1.9999999999999514,-4.8627768478581856e-14
"[0.5623534090613251, 4.536183806757618]",7,0,"[0.8862287346816514, 4.604970103673372]",-1.9999999999891451,-1.085487255636508e-11
"[0.10374153885699955, 5.018745921487388]",11,0,"[0.8862270435486842, 5.81137863244338]",-1.9999999999996156,-3.843592111252292e-13


Usando o método de minimização quase newton with the search method aurea


X0,Iter.,Search Iter.,X Opt.,Opt. Value,Error
"[-1.8385400365595483, 4.359940932133885]",8,1,"[-1.9816604660037758, 4.250174318811837]",-1.9999999688585905,-3.114140945115196e-08
"[8.928601514360016, 3.3197980530117723]",8,1,"[8.90645064444408, 3.4323512379640753]",-1.999999872588146,-1.2741185395626076e-07
"[0.7630828937395717, 7.7991879224011464]",9,1,"[0.886220029505756, 7.674949288969182]",-1.9999999996051645,-3.948354976301971e-10
"[0.5623534090613251, 4.536183806757618]",9,1,"[0.8862407325822685, 4.604967652994412]",-1.9999999988569568,-1.1430432156345205e-09
"[0.10374153885699955, 5.018745921487388]",10,1,"[0.8862115353261343, 5.243005271543596]",-1.9999999708439322,-2.9156067782309947e-08


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = y = np.arange(-3.0, 3.0, 0.05)
X, Y = np.meshgrid(x, y)
zs = np.array(getFunction([np.ravel(X), np.ravel(Y)]))
Z = zs.reshape(X.shape)

ax.plot_surface(X, Y, Z)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()